In [1]:
!where python



C:\Users\usech\anaconda3\envs\TRABAJOSU\python.exe
C:\Users\usech\anaconda3\python.exe


In [1]:

import pandas as pd
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report

DATASET: svm_300k_3_13.csv

In [11]:
dt13 = pd.read_csv("C:\\Users\\usech\\Downloads\\svm_300k_3_13.csv")


In [12]:
dt13.head()

,feature_0,feature_1,feature_2,target
0,0.638212,19.497496,20.964036,2
1,-4.583374,-10.027067,-35.861857,4
2,10.978062,19.480408,-6.396739,5
3,-5.419479,14.517298,13.551027,0
4,0.437531,-1.075047,17.261007,6


In [13]:
dt13.info()

display(dt13.describe())

display(dt13.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   feature_0  300000 non-null  float64
 1   feature_1  300000 non-null  float64
 2   feature_2  300000 non-null  float64
 3   target     300000 non-null  int64  
dtypes: float64(3), int64(1)
memory usage: 9.2 MB


,feature_0,feature_1,feature_2,target
count,300000.000000,300000.000000,300000.000000,300000.000000
mean,-1.701744,-2.250949,0.333825,3.500000
std,14.012792,14.432184,13.992047,2.291292
min,-66.218845,-68.535886,-61.317514,0.000000
25%,-11.189971,-12.129994,-9.111242,1.750000
50%,-1.704282,-2.467849,0.321953,3.500000
75%,7.752403,7.418683,9.779677,5.250000
max,61.280661,67.242381,61.474699,7.000000


feature_0    0
feature_1    0
feature_2    0
target       0
dtype: int64

In [14]:
dt13['target'].value_counts()

target
2    37500
4    37500
5    37500
0    37500
6    37500
7    37500
1    37500
3    37500
Name: count, dtype: int64

Fue necesario hacer una muestra del dataset.  El dataset original tiene 300 mil registros, por lo que a la hora de entrenar y evaluar el modelo duro cargando mas de una hora, por lo que tuvimos que detenerlo, probamos evaluandar cada parametro con diferentes valores y aun asi el tiempo computacional era mayor de una hora. Por lo tanto el ultimo intento fue hacer una muestra del dataset el cual resulto ser la solución al problema. El tamaño de la muestra contiene el 10% de los registros del dataset original y aun asi tomo mucho tiempo. (Se usaron todos los nucleos del procesador disponibles).

In [16]:
dt13_muestra = dt13.sample(n=30000, random_state=42)


In [17]:

# Separas features y target
X = dt13_muestra[['feature_0', 'feature_1', 'feature_2']]
y = dt13_muestra['target']

# Divides datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalas solo las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:


# 1. Definir la grilla de parámetros
param_grid = {
    'C': [0.1, 1],
    'gamma': [0.1, 0.01],
    'kernel': ['linear', 'rbf']
}

# 2. Función de entrenamiento y evaluación
def train_and_evaluate(model, model_name):
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"\n Modelo: {model_name}")
    print(" Accuracy:", accuracy_score(y_test, y_pred))
    print(" Reporte de clasificación:\n", classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred)





ENTRENAMIENTO 0NE VS ONE 

In [19]:

svc = SVC()
grid_ovo = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
ovo = OneVsOneClassifier(grid_ovo)
accuracy_ovo = train_and_evaluate(ovo, "SVM One-vs-One")



 Modelo: SVM One-vs-One
 Accuracy: 0.2848333333333333
 Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.29      0.37      0.32       736
           1       0.26      0.25      0.25       767
           2       0.28      0.39      0.32       731
           3       0.35      0.49      0.41       734
           4       0.31      0.39      0.35       756
           5       0.23      0.16      0.19       787
           6       0.17      0.05      0.07       717
           7       0.25      0.19      0.22       772

    accuracy                           0.28      6000
   macro avg       0.27      0.29      0.27      6000
weighted avg       0.27      0.28      0.27      6000



In [24]:
for i, est in enumerate(ovo.estimators_):
    print(f"Clasificador {i+1}: {est.best_params_}")



Clasificador 1: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clasificador 2: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clasificador 3: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clasificador 4: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 5: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clasificador 6: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 7: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 8: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clasificador 9: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 10: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clasificador 11: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 12: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 13: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clasificador 14: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clasificador 15: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clasificador 16: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clasificador 17: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'

In [25]:
from collections import Counter

# Recolecta todos los parámetros
all_params = [est.best_params_ for est in ovo.estimators_]

# Muestra la combinación más común
most_common = Counter(tuple(sorted(p.items())) for p in all_params).most_common(1)
print("Parámetros más comunes entre clasificadores One-vs-One:")
print(dict(most_common[0][0]))


Parámetros más comunes entre clasificadores One-vs-One:
{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}


In [20]:

grid_ovr = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
ovr = OneVsRestClassifier(grid_ovr)
accuracy_ovr = train_and_evaluate(ovr, "SVM One-vs-Rest")


 Modelo: SVM One-vs-Rest
 Accuracy: 0.16666666666666666
 Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.16      0.08      0.11       736
           1       0.21      0.13      0.16       767
           2       0.12      0.38      0.19       731
           3       0.30      0.43      0.35       734
           4       0.00      0.00      0.00       756
           5       0.11      0.02      0.03       787
           6       0.14      0.23      0.17       717
           7       0.14      0.09      0.11       772

    accuracy                           0.17      6000
   macro avg       0.15      0.17      0.14      6000
weighted avg       0.15      0.17      0.14      6000



C:\Users\usech\anaconda3\envs\TRABAJOSU\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usech\anaconda3\envs\TRABAJOSU\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\usech\anaconda3\envs\TRABAJOSU\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [27]:
print("Mejores parámetros para cada clase (One-vs-Rest):")
for i, est in enumerate(ovr.estimators_):
    print(f"Clase {i}: {est.best_params_}")


Mejores parámetros para cada clase (One-vs-Rest):
Clase 0: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 1: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 2: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 3: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 4: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 5: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 6: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 7: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}


In [28]:


# Recolecta todos los parámetros
all_params = [est.best_params_ for est in ovr.estimators_]

# Muestra la combinación más común
most_common = Counter(tuple(sorted(p.items())) for p in all_params).most_common(1)
print("Parámetros más comunes entre clasificadores One-vs-Rest:")
print(dict(most_common[0][0]))

Parámetros más comunes entre clasificadores One-vs-One:
{'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}


ENTRENAMIENTO 0NE VS REST 

Resultados para el dataset svm_300k_3_13.csv: 
El modelo SVM con estrategia One-vs-One fue claramente mejor en todas las métricas:
-Mayor accuracy.
-Mejor balance en precisión y recall entre clases.
-Aunque no es un rendimiento espectacular (aún lejos del 100%), es significativamente mejor que el One-vs-Rest en este caso.

In [ ]:
#DATASET svm_30k_10_16.csv

In [2]:
dt30 = pd.read_csv("C:\\Users\\usech\\Downloads\\svm_30k_10_16.csv")


In [3]:
dt30.head()

,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,target
0,-51.160673,26.035477,7.561852,-8.092129,-7.717897,1.696000,-12.957637,-1.023136,-0.827129,-51.630709,1
1,-35.515463,-0.513442,-1.594814,-13.581097,34.308047,0.267119,-18.265009,21.534120,11.129230,14.069917,6
2,-2.773373,32.023227,-3.844163,-12.878477,-26.217124,11.329399,-12.848427,-18.847647,12.103943,-13.504171,3
3,-25.633494,25.904482,-32.943225,-24.349416,-33.213071,11.264258,-0.465405,-8.031750,0.179893,0.052532,1
4,-33.149247,-36.844385,-12.259335,22.637652,-3.986942,-1.657094,7.570057,23.744319,5.052129,18.832912,4


In [4]:
dt30.info()
display(dt30.describe())
display(dt30.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30000 entries, 0 to 29999
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   feature_0  30000 non-null  float64
 1   feature_1  30000 non-null  float64
 2   feature_2  30000 non-null  float64
 3   feature_3  30000 non-null  float64
 4   feature_4  30000 non-null  float64
 5   feature_5  30000 non-null  float64
 6   feature_6  30000 non-null  float64
 7   feature_7  30000 non-null  float64
 8   feature_8  30000 non-null  float64
 9   feature_9  30000 non-null  float64
 10  target     30000 non-null  int64  
dtypes: float64(10), int64(1)
memory usage: 2.5 MB


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,target
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,-0.381219,-0.075634,-0.656105,2.901995,-1.044810,1.741902,-4.376453,-1.084179,-1.500626,-2.321567,3.500000
std,17.015133,17.256537,17.762118,16.960943,16.915369,16.848274,16.559829,17.073690,16.668229,16.994324,2.291326
min,-66.455872,-67.052971,-70.420806,-69.476492,-69.403635,-69.768254,-70.540333,-67.392770,-78.976411,-61.717813,0.000000
25%,-11.874222,-11.910839,-12.652520,-8.678437,-12.511855,-9.512350,-15.541589,-12.661651,-12.799525,-13.860247,1.750000
50%,-0.371719,-0.194001,-0.643129,3.002527,-1.217165,1.903634,-4.500999,-1.118787,-1.372115,-2.385987,3.500000
75%,11.156697,11.671624,11.376898,14.483297,10.473894,13.062201,6.793469,10.351066,9.747080,9.144365,5.250000
max,67.496365,70.657990,64.285212,81.971546,61.965336,64.811618,61.225890,70.143326,59.667614,77.247600,7.000000


feature_0    0
feature_1    0
feature_2    0
feature_3    0
feature_4    0
feature_5    0
feature_6    0
feature_7    0
feature_8    0
feature_9    0
target       0
dtype: int64

In [5]:
dt30['target'].value_counts()

target
1    3750
6    3750
3    3750
4    3750
2    3750
5    3750
7    3750
0    3750
Name: count, dtype: int64

In [29]:

# Separas features y target
X = dt30.drop('target', axis=1)
y = dt30['target']

# Divides datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalas solo las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [30]:

# 1. Definir la grilla de parámetros
param_grid = {
    'C': [0.1, 1],
    'gamma': [0.1, 0.01],
    'kernel': ['linear', 'rbf']
}

# 2. Función de entrenamiento y evaluación
def train_and_evaluate(model, model_name):
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    print(f"\n Modelo: {model_name}")
    print(" Accuracy:", accuracy_score(y_test, y_pred))
    print(" Reporte de clasificación:\n", classification_report(y_test, y_pred))
    return accuracy_score(y_test, y_pred)

ENTRENAMIENTO 0NE VS ONE 

In [31]:

svc = SVC()
grid_ovo = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
ovo = OneVsOneClassifier(grid_ovo)
accuracy_ovo = train_and_evaluate(ovo, "SVM One-vs-One")


 Modelo: SVM One-vs-One
 Accuracy: 0.4206666666666667
 Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.44      0.43      0.44       758
           1       0.45      0.48      0.46       730
           2       0.36      0.32      0.34       767
           3       0.37      0.41      0.39       704
           4       0.37      0.34      0.35       741
           5       0.46      0.52      0.49       749
           6       0.45      0.46      0.45       791
           7       0.44      0.41      0.42       760

    accuracy                           0.42      6000
   macro avg       0.42      0.42      0.42      6000
weighted avg       0.42      0.42      0.42      6000



In [34]:
print("Mejores parámetros para cada clase (One-vs-Rest):")
for i, est in enumerate(ovo.estimators_):
    print(f"Clase {i}: {est.best_params_}")


Mejores parámetros para cada clase (One-vs-Rest):
Clase 0: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 1: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 2: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 3: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 4: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 5: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 6: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 7: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 8: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 9: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 10: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 11: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 12: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 13: {'C': 1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 14: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 15: {'C': 0.1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 16: {'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 17: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Clase 18: {'

In [35]:
# Recolecta todos los parámetros
all_params = [est.best_params_ for est in ovo.estimators_]

# Muestra la combinación más común
most_common = Counter(tuple(sorted(p.items())) for p in all_params).most_common(1)
print("Parámetros más comunes entre clasificadores One-vs-One:")
print(dict(most_common[0][0]))

Parámetros más comunes entre clasificadores One-vs-One:
{'C': 0.1, 'gamma': 0.01, 'kernel': 'rbf'}


ENTRENAMIENTO 0NE VS REST 

In [32]:

grid_ovr = GridSearchCV(svc, param_grid, cv=3, n_jobs=-1)
ovr = OneVsRestClassifier(grid_ovr)
accuracy_ovr = train_and_evaluate(ovr, "SVM One-vs-Rest")


 Modelo: SVM One-vs-Rest
 Accuracy: 0.37833333333333335
 Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.40      0.39      0.39       758
           1       0.40      0.43      0.42       730
           2       0.26      0.14      0.18       767
           3       0.35      0.39      0.37       704
           4       0.33      0.30      0.32       741
           5       0.42      0.54      0.47       749
           6       0.40      0.44      0.42       791
           7       0.38      0.39      0.39       760

    accuracy                           0.38      6000
   macro avg       0.37      0.38      0.37      6000
weighted avg       0.37      0.38      0.37      6000



In [37]:
print("Mejores parámetros para cada clase (One-vs-Rest):")
for i, est in enumerate(ovr.estimators_):
    print(f"Clase {i}: {est.best_params_}")

Mejores parámetros para cada clase (One-vs-Rest):
Clase 0: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 1: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 2: {'C': 0.1, 'gamma': 0.1, 'kernel': 'linear'}
Clase 3: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 4: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 5: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 6: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}
Clase 7: {'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


In [38]:
# Recolecta todos los parámetros
all_params = [est.best_params_ for est in ovr.estimators_]

# Muestra la combinación más común
most_common = Counter(tuple(sorted(p.items())) for p in all_params).most_common(1)
print("Parámetros más comunes entre clasificadores One-vs-Rest:")
print(dict(most_common[0][0]))

Parámetros más comunes entre clasificadores One-vs-Rest:
{'C': 1, 'gamma': 0.1, 'kernel': 'rbf'}


Resultados dataset : El modelo One-vs-One sigue siendo superior en todas las métricas.
En particular, mejora el rendimiento en clases como 2 y 4, donde el One-vs-Rest tiene más problemas.
La consistencia entre precision, recall y f1-score también es mejor con One-vs-One.



Observación general: 
Si el dataset está balanceado:
Ambos métodos (OvO y OvR) tienen la misma oportunidad de aprender bien.
La ventaja del One-vs-One no viene del balanceo, sino de que:
Cada clasificador maneja solo 2 clases, es decir fronteras más simples.
Se evitan los problemas de clases dominantes en un clasificador multiclase. Puede capturar mejor patrones finos entre pares de clases.

Si el dataset estuviera desbalanceado:
One-vs-Rest sufre más, porque el clasificador para cada clase compite contra todas las demás, las clases mayoritarias tienden a dominar.One-vs-One es más robusto en este caso.
